In [ ]:
# some simple commands

In [ ]:
import torch


In [ ]:
x = torch.tensor([1., 2.])
y = torch.tensor([3., 2.])
x

In [ ]:
x * y

In [ ]:
x = torch.tensor([
                  [1., 2.], 
                  [3., 2.], 
                  [0., 4.],
                 ])
print(x[0][1])
x[0][1].item()

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
torch

In [ ]:
y = torch.tensor([3., 2.], device="cuda")


In [8]:
# example with mnist


In [69]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F

In [16]:
train = pd.read_csv("../data/mnist/train.csv")

In [17]:
train_labels = train.label.values
train_data = train.drop(columns="label").values.reshape(len(train_labels), 1, 28, 28)

In [18]:
train_data.shape

(42000, 1, 28, 28)

In [19]:
# reshape for a NN

In [21]:
# pytorch requires features to be floats for gradients
y = torch.tensor(train_labels).long() # long -> 64bit integers
# enforce labels as integers
X = torch.tensor(train_data.astype(float)) # 32bit floating-point numbers 



In [22]:
# build neural network

784

In [29]:
train_labels

array([1, 0, 1, ..., 7, 6, 9])

In [36]:
num_labels = len(set(train_labels))
img_height = 28
img_width = 28 

In [70]:
class MNISTClassifier(nn.Module):
    
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        s_in = img_height * img_width
        s2 = 392
        s3 = 196
        s4 = 98
        s_out = num_labels
        self.fc1 = nn.Linear(s_in, s2) # fully connected
        self.fc2 = nn.Linear(s2, s3)
        self.fc3 = nn.Linear(s3, s4)
        self.fc4 = nn.Linear(s4, s_out)
        # regularise NN to avoid overfitting
        self.dropout = nn.Dropout(p=0.2) # 20% of nodes will be randomly 
                                         # not used during training iterations 
                                         # makes network more robust since each node
                                          # will not be used in every iteration.
                    
        def forward(self, X):
            """forward pass"""
            x = x.view(x.shape[0], -1) # flatten img size to one long vector
            x = self.dropout(F.relu(self.fc1(x)))
            x = self.dropout(F.relu(self.fc2(x)))
            x = self.dropout(F.relu(self.fc3(x)))
            x = F.log_softmax(self.fc4(x), dim=1)
            return x
            
            


In [72]:
model = MNISTClassifier()
loss_function = nn.NLLLoss() # negative log likelyhood -> loss = -log(y)
opt = optim.Adam(model.parameters(), lr=0.001)

In [73]:
# Training
